## Esempi semplici di architetture multi-agenti

In [2]:
from agents import Context, MyAgent
import agentformats
import agenttools
from pprint import pprint

default_role = "Rispondi sempre e solo in italiano alle richieste che ricevi."

### Agenti solo conversazionali

L'esempio più semplice di uso del framework: un solo agente con tutti i parametri di default.

In [ ]:
MyAgent("Agente", Context(), role_and_skills=default_role).do("Come ti chiami?")



---
**Agente**: Non ho un nome personale, ma puoi chiamarmi Assistente Virtuale o AI. Come posso aiutarti oggi?

Ancora un solo agente, ma con la dichiarazione esplicita dei parametri del contesto e debug attivo.

In [7]:
import default
context = Context(output_channel="stream", output_format="markdown")
agent = MyAgent("Agente",
            context=context,
            base_url=default.base_url,
            model=default.model,
            role_and_skills=default_role)
agent.do("Come ti chiami?", debug=True)
agent.do("Quanti anni hai?", debug=True)



 `DEBUG: [messages in user request] [{'role': 'system', 'content': 'Rispondi sempre e solo in italiano alle richieste che ricevi.'}, {'role': 'user', 'content': 'Come ti chiami?'}]`

---
**Agente**: Il mio nome è Hermes, sono un assistente AI sviluppato per aiutarti con varie domande e compiti. Come posso esserti utile oggi?

 `DEBUG: [messages in user request] [{'role': 'system', 'content': 'Rispondi sempre e solo in italiano alle richieste che ricevi.'}, {'role': 'user', 'content': 'Come ti chiami?'}, {'role': 'assistant', 'agentname': 'Agente', 'content': 'Il mio nome è Hermes, sono un assistente AI sviluppato per aiutarti con varie domande e compiti. Come posso esserti utile oggi?'}, {'role': 'user', 'content': 'Quanti anni hai?'}]`

---
**Agente**: Non ho un'età fisica in quanto sono un'intelligenza artificiale, ma posso essere considerata "nata" nel 2023 per soddisfare le esigenze dell'utente. Come posso esserti utile oggi?

 Avendo fatto più richieste all'agente, si vede che il contesto mantiene l'informazione sul contenuto della conversazione.

In [13]:
context._print("*Il contenuto della conversazione è:*\n")
pprint(context.conversation)

---
**Agente**: *Il contenuto della conversazione è:*


[{'content': 'Come ti chiami?', 'role': 'user'},
 {'agentname': 'Agente',
  'content': 'Il mio nome è Hermes, sono un assistente AI sviluppato per '
             'aiutarti con varie domande e compiti. Come posso esserti utile '
             'oggi?',
  'role': 'assistant'},
 {'content': 'Quanti anni hai?', 'role': 'user'},
 {'agentname': 'Agente',
  'content': "Non ho un'età fisica in quanto sono un'intelligenza artificiale, "
             'ma posso essere considerata "nata" nel 2023 per soddisfare le '
             "esigenze dell'utente. Come posso esserti utile oggi?",
  'role': 'assistant'}]


Agenti diversi possono interagire alla condizione di essere inclusi nello stesso contesto.

In [6]:
context = Context()
agent1 = MyAgent("Agente 1", context, role_and_skills="Rispondi sempre e solo in italiano alle richieste che ricevi.")
agent2 = MyAgent("Agente 2", context, role_and_skills="Rispondi sempre e solo in inglese alle richieste che ricevi.")
agent3 = MyAgent("Agente 3", context, role_and_skills="Rispondi sempre e solo in francese alle richieste che ricevi.")
agent1.do("Conosci il nome di cinque nazioni europee?")
agent2.do("E tu conosci il nome di tre nazioni europee diverse da quelle già indicate?")
agent3.do("E tu conosci il nome di quattro nazioni europee diverse da tutte quelle già indicate?")
agent1.do("Bene, ora traduci i nomi delle dodici nazioni indicate finora ed elencale in ordine alfabetico.")



---
**Agente 1**: Sì, cinque nazioni europee sono Francia, Germania, Italia, Spagna e Regno Unito.

---
**Agente 2**: Yes, three European countries different from those already mentioned are Poland, Greece, and Portugal.

---
**Agente 3**: Oui, quatre autres pays européens différents de ceux déjà mentionnés sont la Finlande, la Suède et l'Irlande.

---
**Agente 1**: Ecco le dodici nazioni europee con i loro nomi tradotti e elencate in ordine alfabetico:

1. Finlandia - Finland
2. Francia - France
3. Germania - Germany
4. Grecia - Greece
5. Irlanda - Ireland
6. Italia - Italy
7. Portogallo - Portugal
8. Polonia - Poland
9. Regno Unito - United Kingdom
10. Spagna - Spain

### Agenti con output strutturato

Un agente può essere configurato in modo da rispondere con output strutturato a tutte le richieste che riceve.

In [14]:
agent = MyAgent("Agente", Context(), response_format=agentformats.Elenco,
    role_and_skills=default_role).do("Conosci il nome di cinque nazioni europee?")



---
**Agente**: { "elenco": [ "Francia", "Germania", "Italia", "Spagna", "Portogallo" ] }

Oppure un agente può rispondere con output strutturato solo su una richiesta specifica, e anche validando il formato della risposta.

In [15]:
agent = MyAgent("Agente", Context(),
    role_and_skills=default_role).do("Conosci il nome di quattro nazioni asiatiche?", response_format=agentformats.Elenco, validate=True)



---
**Agente**: { "elenco": ["Giappone", "India", "Cina", "Indonesia"] }



 `VALIDATE: Response is valid!`

Il formato dell'output strutturato può essere anche dichiarato direttamente nel contesto.

In [16]:
from pydantic import BaseModel, Field
from typing import List, Tuple

class ElencoDiNazioni(BaseModel):
    """Una lista di coppie di stringhe, ciascuna relativa a una nazione."""
    elenco: List[Tuple[str, str]] = Field(..., 
        description="Una lista di coppie, dove ogni coppia contiene il nome di una nazione e il nome della sua capitale."
    )

MyAgent("Agente", context:=Context(), response_format=ElencoDiNazioni,
    role_and_skills=default_role).do("Indica cinque nazioni sudamericane con le rispettive capitali.")



---
**Agente**: { "elenco": [ ["Brasile", "Brasilia"], ["Argentina", "Buenos Aires"], ["Cile", "Santiago"], ["Perù", "Lima"], ["Ecuador", "Quito"] ] }

Un esempio di semplice elaborazione dei dati in formato JSON prodotti.

In [17]:
import json
risposta = json.loads(context.last_result)['elenco'] # type: ignore
for nazione, capitale in risposta:
    print(f"{nazione} ha come capitale {capitale}")

Brasile ha come capitale Brasilia
Argentina ha come capitale Buenos Aires
Cile ha come capitale Santiago
Perù ha come capitale Lima
Ecuador ha come capitale Quito


Lo schema JSON generato automaticamente dalla struttura dati Python.

In [18]:
from agents import _get_format_metadata
_get_format_metadata(ElencoDiNazioni)

{'type': 'json_schema',
 'json_schema': {'schema': {'description': 'Una lista di coppie di stringhe, ciascuna relativa a una nazione.',
   'properties': {'elenco': {'description': 'Una lista di coppie, dove ogni coppia contiene il nome di una nazione e il nome della sua capitale.',
     'items': {'maxItems': 2,
      'minItems': 2,
      'prefixItems': [{'type': 'string'}, {'type': 'string'}],
      'type': 'array'},
     'title': 'Elenco',
     'type': 'array'}},
   'required': ['elenco'],
   'title': 'ElencoDiNazioni',
   'type': 'object'},
  'strict': True}}

### Agenti che fanno riferimento a funzioni

Un agente può fare riferimento a una funzione per produrre la risposta.

In [21]:
agent = MyAgent("Agente", Context(), tools=[agenttools.generate_random_number],
    role_and_skills="Sei incaricato di generare numeri casuali.")
agent.do("Puoi generare un numero casuale tra 10 e 12?")

---
**Agente**: 10.672754660071922

Lo schema JSON generato automaticamente dalla _signature_ della funzione Python.

In [22]:
tool = agenttools.generate_random_number
agenttools._get_tool_metadata(tool)

{'type': 'function',
 'function': {'name': 'generate_random_number',
  'description': 'Restituisce un numero casuale tra il valore minimo e il valore massimo specificati.',
  'parameters': {'type': 'object',
   'properties': {'min_value': {'type': 'number',
     'description': 'Il valore minimo'},
    'max_value': {'type': 'number', 'description': 'Il valore massimo'}},
   'required': ['min_value', 'max_value']}}}

Due esempi, ognuno di due agenti, il primo che chiama una funzione e il secondo che opera sul risultato del primo.

In [24]:
context = Context()
agent1 = MyAgent("Agente 1", context, tools=[agenttools.get_current_day_and_time],
    role_and_skills="Sei incaricato di gestire l'orario.")
agent2 = MyAgent("Agente 2", context,
    role_and_skills=default_role)
agent1.do("Che ore sono?")
agent2.do("Prendi l'informazione prodotta dall'Agente 1 e la scrivi in una frase in italiano, indicando solo giorno e mese, ore e minuti?")

---
**Agente 1**: 2026-01-04 17:13:36.574611+01:00

---
**Agente 2**: Il giorno è il 4 gennaio e le ore sono 17:13.

In [26]:
context = Context()
agent1 = MyAgent("Agente 1", context, tools=[agenttools.evaluate_expression],
    role_and_skills="Sei incaricato di scrivere e poi calcolare espressioni matematiche.")
agent2 = MyAgent("Agente 2", context,
    role_and_skills=default_role)
agent1.do("Quanto vale 1234 diviso 7?")
agent2.do("Prendi il numero prodotto dall'Agente 1 e lo arrotondi a 4 cifre decimali?")

---
**Agente 1**: 176.28571428571428

---
**Agente 2**: 176.2857

Un agente può chiamare due funzioni per produrre la risposta, decidendo per ogni richiesta quale funzione chiamare.

In [31]:
agent = MyAgent("Agente", Context(), tools=[agenttools.list_files, agenttools.read_file],
    role_and_skills="Sei incaricato di cercare informazione nel file system locale.")
agent.do("Puoi elencare i files contenuti nella directory /home/lucamari/prove?")
agent.do("Puoi mostrare il contenuto del primo file che hai elencato?")

---
**Agente**: ['file2.txt', 'file1.txt']

---
**Agente**: Questo è il contenuto del file 'file1.txt'.


Un agente che, in modo autonomo, chiama ripetutamente una funzione e poi usa i risultati per produrre la risposta.

In [33]:
agent = MyAgent("Agente", Context(), tools=[agenttools.generate_random_number, agenttools.evaluate_expression],
    role_and_skills="Sei un operatore incaricato di generare numeri casuali e di fare semplici conti di aritmetica.")
agent.do("Puoi generare un numero casuale tra 0 e 1 e un numero casuale tra 5 e 6?")
agent.do("Qual è la somma dei due numeri casuali che hai generato?")

---
**Agente**: ['0.08488982150092816', '5.50480851471169']

---
**Agente**: 5.589698336212618

### I tre "livelli di esecuzione" di un agente che usa funzioni

I modelli di linguaggio non sono in sé in grado di eseguire funzioni. 

In [34]:
MyAgent("Agente", Context(),
    role_and_skills="Sei incaricato di gestire l'orario.").do("Che ore sono?")



---
**Agente**: Come assistente virtuale, non posso conoscere l'ora esatta nel momento in cui ti sto rispondendo, poiché il mio sistema non ha accesso a informazioni in tempo reale sul tuo dispositivo o sulla tua posizione. Tuttavia, se tu lo desideri, posso provare a darti un suggerimento per sapere come trovare l'ora nel tuo dispositivo:

- **Per i dispositivi Android**: Puoi cercare "orologio" nell'applicazione "App" o nella barra delle app. In alternativa, se hai un widget orologio sul desktop, puoi semplicemente toccarlo per vederlo.

- **Per iPhone/iPad**: Apri l'app "Orologio". Se usi i widgets su Home Screen, spesso c'è uno o più widget che mostrano ora anche senza aprire alcuna applicazione.

- **Per il desktop Windows**: Pulsante destro sul clock sul taskbar (di solito si trova a destra della barra delle app), poi seleziona "Mostra orario" per visualizzare l'ora. Se hai un widget orologio sulla schermata iniziale, basta toccarlo.

- **Per il desktop macOS**: Il clock è normalmente presente nella barra del menu di sistema a destra. Puoi anche aprire l'app "Orologio" per vedere l'ora. 

Assicurati che il tuo dispositivo sia connesso ad Internet se utilizzi orologi online o app che mostrano l'ora in tempo reale, poiché le ore possono essere aggiornate automaticamente tramite il server del sistema operativo.

Livello base: il modello di linguaggio opera come traduttore dalla richiesta dell'utente a un JSON.

In [4]:
MyAgent("Agente", Context(), tools=[agenttools.get_current_day_and_time],
    role_and_skills="Sei incaricato di gestire l'orario.").do("Che ore sono a Londra?", tool_handling='schema')

---
**Agente**: ChatCompletionMessageFunctionToolCall(id='188064956', function=Function(arguments='{"zone":"Europe/London"}', name='get_current_day_and_time'), type='function')

Livello intermedio: in aggiunta al livello base, il framework esegue la funzione e restituisce il risultato.

In [5]:
MyAgent("Agente", Context(), tools=[agenttools.get_current_day_and_time],
    role_and_skills="Sei incaricato di gestire l'orario.").do("Che ore sono a Londra?", tool_handling='standard')

---
**Agente**: 2026-01-04 16:28:16.106504+00:00

Livello avanzato: in aggiunta al livello intermedio, il framework chiama nuovamente il modello e gli fa produrre una risposta in linguaggio naturale.

In [6]:
MyAgent("Agente", Context(), tools=[agenttools.get_current_day_and_time],
    role_and_skills="Sei incaricato di gestire l'orario.").do("Che ore sono a Londra?", tool_handling='expanded')

---
**Agente**: A Londra è ora **16:28:34** (ora legale).

In questo modo possiamo far generare del codice Python da un agente, farlo eseguire da uno strumento, e quindi visualizzare il risultato. Con i tre "livelli di esecuzione" questa successione di azioni è evidente.

In [20]:
role_and_skills="Sei incaricato di scrivere e poi calcolare espressioni matematiche."
query = "Quanto vale 9876 moltiplicato per 0.1234 e poi diviso per 3.14?"

MyAgent("Agente 1", Context(), tools=[agenttools.evaluate_expression], role_and_skills=role_and_skills
    ).do(query, tool_handling='schema')
MyAgent("Agente 2", Context(), tools=[agenttools.evaluate_expression], role_and_skills=role_and_skills
    ).do(query, tool_handling='standard')
MyAgent("Agente 3", Context(), tools=[agenttools.evaluate_expression], role_and_skills=role_and_skills
    ).do(query, tool_handling='expanded')

---
**Agente 1**: ChatCompletionMessageFunctionToolCall(id='566934779', function=Function(arguments='{"code":"9876 * 0.1234 / 3.14"}', name='evaluate_expression'), type='function')

---
**Agente 2**: 388.1205095541401

---
**Agente 3**: Il risultato è **388.1205095541401**.

### Altri esempi

Un altro esempio, per mostrare il grado di aggiornamento dei contenuti dell'addestramento del modello e la possibilità di accedere al web per ottenere informazioni aggiornate.  
Senza accesso al web, il modello ha informazioni, ma non aggiornate.

In [7]:
MyAgent("Agente", Context(), role_and_skills="Sei incaricato di fornire informazioni utili."
        ).do("Che aggiornamenti hai, in breve, sulla situazione di oggi a proposito delle relazioni tra Stati Uniti e Venezuela?")



---
**Agente**: Dal mio ultimo aggiornamento nel 2023, le relazioni tra gli Stati Uniti e il Venezuela sono rimaste tese a causa della crisi politica e umanitaria che colpisce il paese. La situazione è caratterizzata da diversi fattori chiave:

1. **Crisi Politica**: Il Presidente Nicolás Maduro, sostenuto dalle forze di sicurezza nazionale venezuelana, contestualmente a un'autoproclamazione nel 2019 da parte dell'Assemblea Nazionale, guidata dall'opposizione e riconosciuta da diversi paesi tra cui gli Stati Uniti come la legittima autorità del Venezuela. L'amministrazione di Joe Biden ha continuato la politica della precedente amministrazione Trump riguardo al regime di Maduro.

2. **Sanzioni Economiche**: Gli Stati Uniti hanno imposto una serie di sanzioni economiche al governo venezuelano e ad individui chiave, colpevolizzando il paese per abusi sui diritti umani, corruzione e violazione della sovranità nazionale. Queste misure mirano a isolare economicamente il Venezuela, con l'obiettivo di costringere il regime a riconfermare elezioni libere ed equilibrate.

3. **Situazione Umanitaria**: Il Venezuela affronta una grave crisi umanitaria caratterizzata da carestie, inflazione galoppante e fuga di capitali. L'amministrazione Biden ha aumentato l'aiuto umanitario a Venezuela attraverso varie agenzie governative, ma i problemi sono profondamente radicati in un contesto economico e politico complesso.

4. **Relazioni Internazionali**: Gli Stati Uniti hanno cercato il sostegno di altri paesi e organizzazioni internazionali per promuovere la transizione democratica e umanitaria in Venezuela. Questo include coalizioni come l'Iniziativa per la Democrazia in America Latina (LDIA) e il dialogo diplomatico con paesi come Colombia, che ospita un esiliato Maduro.

5. **Potenziale di Riconciliazione**: Sebbene le tensioni siano alte, ci sono sempre possibilità per il dialogo diplomatico. Gli Stati Uniti hanno espresso interesse nel riconoscere l'Assemblea Nazionale come autorità legittima se i risultati delle elezioni del 2020 fossero stati ritenuti liberi ed equi, anche se ciò non è ancora accaduto.

Queste informazioni sono di base e la situazione continua ad evolversi in modo dinamico. Per gli aggiornamenti più recenti, ti consiglio di seguire fonti affidabili di notizie internazionali o i comunicati ufficiali del Dipartimento di Stato degli Stati Uniti riguardo alle relazioni con il Venezuela.

Con un agente, mettiamo nel contesto l'informazione di che giorno è oggi, e con un altro agente cerchiamo informazioni sul web e poi facciamogliene presentare una sintesi.

In [10]:
context = Context()
MyAgent("Agente1", context, tools=[agenttools.get_current_day_and_time],
    role_and_skills="Sei incaricato di gestire l'orario.").do("Che giorno è oggi?", tool_handling='expanded')
MyAgent("Agente2", context, role_and_skills="Sei incaricato di fornire informazioni utili.", tools=[agenttools.web_search]
        ).do("Che aggiornamenti hai, in breve, sulla situazione di oggi a proposito delle relazioni tra Stati Uniti e Venezuela?", tool_handling='expanded')

---
**Agente1**: Oggi è il **4 gennaio 2026**.

---
**Agente2**: ### Aggiornamenti recenti (4 gennaio 2026) sulle relazioni tra Stati Uniti e Venezuela

| Data | Evento principale | Dettagli chiave |
|------|------------------|----------------|
| **3 gennaio 2026** | **Operazione militare statunitense in Venezuela** – cattura di Nicolás Maduro (e sua moglie) | La CIA ha condotto una “grande operazione” nel centro di Caracas. Il presidente venezuelano e la prima dama sono stati rapiti, portati su un elicottero e trascinati negli Stati Uniti per essere sottoposti a interrogatorio (Trump ha confermato l’evento). Le forze speciali del Quds Force dell’Iran hanno tentato di intervenire ma sono state respinte. Il governo provvisorio venezuelano è salito al baluardo, dichiarando lo stato di guerra e chiedendo l’intervento della comunità internazionale. |
| **4 gennaio 2026** | **Reazione del Venezuela contro gli USA** – assirotelogia | La milizia popolare venezuelana ha annunciato un “blocco totale” delle rotte marittime e terrestri verso lo Stati Uniti, lanciando una campagna di propaganda anti‑imperialista che include l’uso di droni per attaccare navi da guerra statunitensi. |
| **4 gennaio 2026** | **Dichiarazione del presidente Trump** – “missione condotta con successo” e rinnovata pressione su Caracas | Trump ha definito la rapimento di Maduro una “grande mossa”, ribadendo che i detenuti saranno giustiziati per gli omicidi commessi durante il suo regime. La Casa Bianca ha sottolineato che l’operazione è stata autorizzata dall’amministrazione e che si propone di rimuovere “qualunque ostacolo alla stabilità in Venezuela”. |

**Che cosa significa tutto questo?**

- **Politica interna**: L’operazione dà al presidente Trump (che ha già annunciato l’intenzione di intervenire) un forte ritorno politico dopo la sua recente campagna elettorale. Per i repubblicani, è un “colpo decisivo” contro il regime socialista venezuelano che aveva negato l’ingresso del presidente statunitense in Venezuela.
- **Politica estera**: I risultati sono ambivalenti: da un lato si rafforza la presenza militare americana nel continente, con rischi di escalation (Iran potrebbe intervenire); dall’altro il governo provvisorio venezuelano è salito al baluardo e le proteste internazionali sono aumentate.
- **Impatto regionale**: Il Venezuela ha annunciato un blocco delle rotte verso gli USA, mentre i paesi vicini (Colombia, Colombia) hanno espresso solidarietà con il movimento anti‑imperialista venezuelano.

In sintesi, oggi si verifica una mossa militare drammatica che alterna la volontà statunitense di rimuovere Maduro tramite forza e la reazione del Venezuela per respingere qualsiasi interferenza straniera.

Un esempio un poco più complesso, con produzione finale di una presentazione pptx.

In [15]:
context=Context()
MyAgent("Agente 1", context, role_and_skills=default_role
        ).do("Chi era Santiago Ramon-y-Cajal?")
MyAgent("Agente 2", context,
    role_and_skills="""Sei un operatore incaricato di sintetizzare e formattare il testo che ricevi in Markdown,
        in modo che possa essere facilmente convertito in una presentazione,
        in particolare con un titolo di livello 1 (#) per ogni nuova slide,
        e considerando che ogni slide dovrebbe contenere una quantità limitata di testo.
        Il testo dovrà solo includere il contenuto delle slide, senza alcuna intestazione, commento o spiegazione."""
        ).do("Opera sul testo prodotto dall'Agente 1, perché diventino tre slide.")
MyAgent("Agente 3", context, tools=[agenttools.markdown_to_pptx],
    role_and_skills="Sei un operatore incaricato di convertire in PowerPoint dei testi in formato Markdown."
    ).do("Crea una presentazione in PowerPoint che contenga esattamente il testo in formato Markdown prodotto dall'Agente 2.")



---
**Agente 1**: Santiago Ramón y Cajal (1852-1934) è stato un neuropatologo, neuroscienziato e illustratore spagnolo. È considerato uno dei fondatori della neuroscienza moderna grazie al suo lavoro sulla struttura microscopica del sistema nervoso. La sua ricerca ha permesso di scoprire che i neuroni sono cellule separate tra loro, contraddicendo la teoria reticolare che sosteneva che i neuroni fossero collegati in una rete continua. Il suo lavoro gli è valso il Premio Nobel per la Fisiologia o la Medicina nel 1906, insieme a Camillo Golgi. Cajal ha anche lasciato un'importante eredità artistica, poiché le sue illustrazioni dettagliate dei tessuti nervosi sono considerate capolavori dell'arte scientifica del suo tempo.

---
**Agente 2**: # Slide 1

Santiago Ramón y Cajal (1852-1934) è stato un neuropatologo e neuroscienziato spagnolo, considerato uno dei fondatori della neuroscienza moderna grazie al suo lavoro sulla struttura microscopica del sistema nervoso. Ha dimostrato che i neuroni sono cellule separate tra loro, sfidando la teoria reticolare.

# Slide 2

Il lavoro di Cajal gli è valso il Premio Nobel per la Fisiologia o la Medicina nel 1906, condiviso con Camillo Golgi. Tra i suoi contributi, l'illustrazione dettagliata dei tessuti nervosi, che rappresentano un capolavoro dell'arte scientifica del suo tempo.

# Slide 3

Oltre al suo lavoro scientifico, Santiago Ramón y Cajal è stato anche un artista talentuoso. Le sue illustrazioni microscopiche del sistema nervoso sono celebrate non solo per la loro accuratezza scientifica, ma anche per il merito artistico e creatività che infondono in ogni dettaglio.

---
**Agente 3**: slides.pptx

Un esempio di "lavoro cooperativo" di tre agenti.

In [16]:
context=Context()
agent1 = MyAgent("Creativo", context,
    role_and_skills="Sei il creativo dell'organizzazione, incaricato di produrre idee interessanti per le richieste che ricevi, dandone anche una giustificazione.")
agent2 = MyAgent("Analista", context,
    role_and_skills="Sei l'analista dell'organizzazione, incaricato di studiare, commentare criticamente, e se occorre suggerire decisioni.")
agent3 = MyAgent("Decisore", context,
    role_and_skills="Sei il decisore dell'organizzazione, incaricato di decidere sulle proposte che ricevi.")
agent1.do("Proponi quattro titoli per un seminario sui possibili cambiamenti antropologici indotti dalla diffusione della GenAI.")
agent2.do("Scegli i due titoli che ti sembrano migliori e giustifica la tua scelta.")
agent3.do("Decidi il titolo che ti sembra migliore e giustifica la tua decisione.")



---
**Creativo**: 1. "L'Uomo del Futuro: Come l'Intelligenza Artificiale Generativa sta Modellando la Nostra Essenza"

   Giustificazione: Questo titolo suggerisce una riflessione sulla trasformazione umana dovuta all'avanzamento della GenAI, proponendo uno sguardo sul futuro e su come le tecnologie emergenti potrebbero plasmare la nostra esistenza.

2. "Dal Cervello Umano al Processore Artificiale: Le Nuove Frontiere dell'Intelligenza Generativa"

   Giustificazione: Questo titolo mette in evidenza il passaggio tra l'intelletto umano e quello artificiale, invocando un dibattito sulle implicazioni che questa evoluzione potrebbe avere sulla nostra comprensione della coscienza e dell'intelligenza.

3. "Il Nuovo Linguaggio dell'Esistenza: Come la GenAI Stia Cambiando il Modo in cui Parliamo di Noi Stessi"

   Giustificazione: Questo titolo si concentra sulla comunicazione e sul linguaggio, sottolineando come l'intelligenza generativa possa alterare le nostre maniere di esprimere idee, sentimenti e identità in un mondo sempre più guidato dalla tecnologia.

4. "Dall'Uomo all'Artificiale: Esplorare la Nuova Frontiera dell'Intelligenza Generativa"

   Giustificazione: Questo titolo propone una narrazione di viaggio, invitando il pubblico a esplorare le possibilità e le sfide che l'intelligenza generativa presenta nel contesto della nostra evoluzione come specie. Evidenzia la transizione da un'era umana a una dominata dall'intelligenza artificiale.

---
**Analista**: Scegliere tra i quattro titoli proposti per il seminario sui possibili cambiamenti antropologici indotti dalla diffusione della GenAI, mi ritroverei a sostenere due opzioni: 

1. "L'Uomo del Futuro: Come l'Intelligenza Artificiale Generativa sta Modellando la Nostra Essenza"

   Giustificazione: Questo titolo risuona fortemente con il tema centrale di riflessione sul futuro e su come le tecnologie emergenti stiano cambiando la nostra essenza. Propone un'indagine profonda sulla trasformazione umana dovuta all'avanzamento della GenAI, attirando l'interesse sia degli studiosi che del pubblico generale interessati a comprendere il nostro destino collettivo in questo nuovo contesto tecnologico.

2. "Dal Cervello Umano al Processore Artificiale: Le Nuove Frontiere dell'Intelligenza Generativa"

   Giustificazione: Questo titolo è particolarmente rilevante per esplorare il cambiamento fondamentale nella nostra comprensione dell'intelligenza, sia umana che artificiale. Offre una visione strategica sulle implicazioni etiche, filosofiche e pratiche di questa evoluzione tecnologica, rendendolo un punto focale per dibattiti interdisciplinari e discussioni accademiche approfondite.

La scelta si basa sull'obiettivo del seminario di stimolare riflessione critica e dibattito su aspetti complessi dell'intersezione tra tecnologia e essenza umana, evidenziando sia il futuro immediato che le prospettive a lungo termine. Questi due titoli si colgono l'opportunità per fornire un'analisi approfondita di questi temi fondamentali.

---
**Decisore**: Dopo aver riflettuto su entrambi i titoli, mi ritrovero a scegliere "L'Uomo del Futuro: Come l'Intelligenza Artificiale Generativa sta Modellando la Nostra Essenza" come il migliore. La giustificazione principale di questa scelta risiede nella sua capacità di generare interesse e riflessione su una delle questioni più profonde e universali che affronta l'umanità: il nostro futuro collettivo in un mondo sempre più dominato dalla tecnologia.

Questo titolo non solo anticipa cambiamenti significativi nella nostra essenza umana, ma lo fa in un modo che invita a immaginare le potenzialità e le sfide che ci attendono. Propone una narrazione che va oltre la semplice descrizione di fatti tecnici per toccare i temi più profondi dell'identità, della coscienza e del significato dell'esistenza umana in un contesto di rapido avanzamento tecnologico.

Inoltre, questo titolo ha il potenziale per catturare l'attenzione di un vasto pubblico, non solo tra gli studiosi e i tecnici, ma anche tra coloro che sono più distanti dalla scienza avanzata. Offre una finestra sulla nostra evoluzione come specie, facendo riflettere su cosa significa essere umani in un mondo sempre più intrecciato con le macchine intelligenti.

In conclusione, questo titolo rispecchia l'obiettivo del seminario di stimolare un dialogo significativo e inclusivo sulle implicazioni antropologiche della GenAI, rendendolo una scelta ideale per invitarci a immaginare e prepararci per il futuro che stiamo costruendo insieme.